<a href="https://colab.research.google.com/github/AdmiralJPJ/ai/blob/main/ML_HW4_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


*Resources used: Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow book by Aurélien Géron, ChatGPT-4o, Google Gemini*

## Applied Machine Learning HW 3 Q1, Group 36, **Jack Tyndall**

In [4]:
import os
from PIL import Image
from torch.utils.data import Dataset
from transformers import CLIPProcessor

class ClipImageDataset(Dataset):
    def __init__(self, root_dir, condition, categories, processor):
        self.processor = processor
        self.categories = categories
        self.image_paths = []
        self.labels = []

        base_path = os.path.join(root_dir, condition)

        for label_idx, category in enumerate(categories):
            for filename in os.listdir(base_path):
                if (filename.lower().endswith(("jpg", "png", "jpeg"))) and (category in filename.lower()):
                    self.image_paths.append(os.path.join(base_path, filename))
                    self.labels.append(label_idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(path).convert("RGB")

        processed = self.processor(images=image, return_tensors="pt")
        processed["pixel_values"] = processed["pixel_values"].squeeze(0)

        return processed["pixel_values"], label, path



In [5]:
from transformers import CLIPModel, CLIPProcessor
from torch.utils.data import DataLoader

categories = ["airplane", "car", "chair", "cup", "dog", "donkey", "duck", "hat"]

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

dataset = ClipImageDataset(
    root_dir="/content/drive/MyDrive/HW3/v0",
    condition="realistic",
    categories=categories,
    processor=processor
)

dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

### **HW3Q1 1.**

Images loaded in, gray scaled, and reduced to 128x128. Each condition is stored in a dict.

Each number of components needed for 95% variance is displayed, and it tracks that the higher quality, more full images, require more components to maintain variance. Blurred images, which had little detail, required the least amount of components, while geons, which appear to be high definition simple 3D renderings, require the most.
